In [388]:
from pygmid import Lookup as lk
import numpy as np
import scipy.constants as sc

In [389]:
n = lk('nfet_03v3.mat')
p = lk('pfet_03v3.mat')

In [390]:
c_load = 30e-12
gm_id_m12 = 10
gm_id_m34 = 5
gm_id_m567 = 5
gm_id_m8 = 5
l_12 = 3
l_34 = 3
l_56 = 3
l_7 = 3
l_8 = 1
f_bw = 10e6 # -3dB bandwidth of the voltage buffer
i_tail_limit = 100e-6
i_bias_in = 80e-6
output_voltage = 1.3
vin_min = 1.6
vin_max = 1.7
vdd_min = 3.3
vdd_max = 3.3

In [391]:
# To make sure PM of 60 degrees
c_comp = (2.2/10) * c_load
print('c_comp (exact) =', round(c_comp/1e-15, 1), 'fF')
c_comp = round(c_comp/1e-15, 0) * 1e-15
print('c_comp (rounded) =', round(c_comp/1e-15, 0), 'fF')

c_comp (exact) = 6600.0 fF
c_comp (rounded) = 6600.0 fF


In [392]:
# Find gm_12
gm_m12 = f_bw * 2* np.pi * c_comp
print('gm12 =', round(gm_m12/1e-3, 4), 'mS')

gm12 = 0.4147 mS


In [393]:
id_m12 = gm_m12 / gm_id_m12
i_tail = 2*id_m12
print('i_tail (exact) =', round(i_tail/1e-6, 1), 'µA')
i_tail = round(i_tail/1e-6, 0) * 1e-6
print('i_tail (rounded) =', round(i_tail/1e-6, 1), 'µA')
id_m12 = i_tail/2
if i_tail < i_tail_limit:
    print('[info] power consumption target is met!')
else:
    print('[info] power consumption target is NOT met!') 

i_tail (exact) = 82.9 µA
i_tail (rounded) = 83.0 µA
[info] power consumption target is met!


In [394]:
gm_gds_m8 = p.lookup('GM_GDS', GM_ID=gm_id_m8, L=l_8, VDS=1.65, VSB=0)

# General rule of thumb, gm of 2nd stage is 10 times gm of 1st stage
gm_m8 = 10 * gm_m12
gds_m8 = gm_m8 / gm_gds_m8

# Output stage current
id_m78 = gm_m8 / gm_id_m8
print('id_m78 (exact) =', round(id_m78/1e-6, 1), 'µA')
id_m78 = round(id_m78/1e-6, 0) * 1e-6
print('id_m78 (rounded) =', round(id_m78/1e-6, 1), 'µA')

id_m78 (exact) = 829.4 µA
id_m78 (rounded) = 829.0 µA


In [395]:
gm_gds_m12 = n.lookup('GM_GDS', GM_ID=gm_id_m12, L=l_12, VDS=1.65, VSB=0)
gm_gds_m34 = p.lookup('GM_GDS', GM_ID=gm_id_m34, L=l_34, VDS=1.65, VSB=0)

gds_m12 = gm_m12 / gm_gds_m12
gm_m34 = gm_id_m34 * i_tail/2
gds_m34 = gm_m34 / gm_gds_m34

gm_gds_m7 = n.lookup('GM_GDS', GM_ID=gm_id_m567, L=l_7, VDS=1.65, VSB=0)
gm_m7 = gm_id_m567 * id_m78
gds_m7 = gm_m7 / gm_gds_m7

# DC gain
a0 = (gm_m12 / (gds_m12 + gds_m34)) * (gm_m8 / (gds_m8 + gds_m7))
print('a0 =', round(20*np.log10(a0), 1), 'dB')

a0 = 101.6 dB


In [396]:
gm_m12 = f_bw * 3 * 4*np.pi*c_load
print('gm12 =', round(gm_m12/1e-3, 4), 'mS')
cgs_m12 = gm_m12/n.lookup('GM_CGS', GM_ID=gm_id_m12, L=l_12, VDS=1.65, VSB=0)
print('cgsm12 =', round(cgs_m12/1e-15, 4), 'fF')

gm12 = 11.3097 mS
cgsm12 = 12225.4549 fF


In [397]:
vgs_m12 = n.look_upVGS(GM_ID=gm_id_m12, L=l_12, VDS=1.65, VSB=0.0)
vgs_m34 = p.look_upVGS(GM_ID=gm_id_m34, L=l_34, VDS=1.65, VSB=0.0) 
vgs_m56 = n.look_upVGS(GM_ID=gm_id_m567, L=l_56, VDS=1.65, VSB=0.0) 
vgs_m7 = n.look_upVGS(GM_ID=gm_id_m567, L=l_7, VDS=1.65, VSB=0.0) 
vgs_m8 = n.look_upVGS(GM_ID=gm_id_m8, L=l_8, VDS=1.65, VSB=0.0) 

print('vgs_12 =', round(float(vgs_m12), 3), 'V')
print('vgs_34 =', round(float(vgs_m34), 3), 'V')
print('vgs_56 =', round(float(vgs_m56), 3), 'V')
print('vgs_7 =', round(float(vgs_m56), 3), 'V')
print('vgs_8 =', round(float(vgs_m56), 3), 'V')

vgs_12 = 0.81 V
vgs_34 = 1.142 V
vgs_56 = 1.029 V
vgs_7 = 1.029 V
vgs_8 = 1.029 V


In [403]:
# calculate all widths
id_w_m12 = n.lookup('ID_W', GM_ID=gm_id_m12, L=l_12, VDS=vgs_m12, VSB=0)
w_12 = id_m12 / id_w_m12
w_12_round = round(w_12, 0)
print('M1/2 W =', round(w_12, 2), 'um, rounded W =', w_12_round, 'um')

id_m34 = id_m12
id_w_m34 = p.lookup('ID_W', GM_ID=gm_id_m34, L=l_34, VDS=vgs_m34, VSB=0)
w_34 = id_m34 / id_w_m34
w_34_round = round(w_34, 0)
print('M3/4 W =', round(w_34, 2), 'um, rounded W =', w_34_round, 'um')

id_w_m6 = n.lookup('ID_W', GM_ID=gm_id_m567, L=l_56, VDS=vgs_m56, VSB=0)
w_6 = i_tail / id_w_m6
w_6_round = round(w_6, 0)
print('M6 W =', round(w_6, 2), 'um, rounded W =', w_6_round, 'um')
w_5 = w_6_round * i_bias_in / i_tail
w_5_round = round(w_5, 0)
print('M5 W =', round(w_5_round, 2), 'um')
w_7 = w_5_round * id_m78 / i_bias_in
w_7_round = round(w_7, 0)
print('M7 W =', round(w_7_round, 2), 'um')


id_w_m8 = p.lookup('ID_W', GM_ID=gm_id_m8, L=l_8, VDS=vgs_m8, VSB=0)
w_8 = id_m78 / id_w_m8
w_8_round = max(round(w_8*2)/2, 0.5)
print('M8 W =', round(w_8, 2), 'um, rounded W =', w_8_round, 'um')

M1/2 W = 62.85 um, rounded W = 63.0 um
M3/4 W = 63.38 um, rounded W = 63.0 um
M6 W = 26.93 um, rounded W = 27.0 um
M5 W = 26.0 um
M7 W = 269.0 um
M8 W = 378.1 um, rounded W = 378.0 um
